## Import Libraries

In [1]:
import numpy    as np
from numpy.testing._private.utils import decorate_methods
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl
import time

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor

## Read CSV file

In [16]:
data = pd.read_csv(r'data/london_merged.csv')
data.head(5)

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [17]:
np.random.seed(0)

## Preprocess the Data

In [18]:
data['year'] = data['timestamp'].apply(lambda row: row[:4])
data['month'] = data['timestamp'].apply(lambda row: row.split('-')[2][:2] )
data['hour'] = data['timestamp'].apply(lambda row: row.split(':')[0][-2:] )

In [19]:
data.drop('timestamp', axis=1, inplace=True)
data.head(3)

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
0,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0,2015,04,00
1,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0,2015,04,01
2,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0,2015,04,02


## Enhance Data

In [20]:
def data_enhancement(data):
    
    gen_data = data
    
    for season in data['season'].unique():
        seasonal_data =  gen_data[gen_data['season'] == season]
        hum_std = seasonal_data['hum'].std()
        wind_speed_std = seasonal_data['wind_speed'].std()
        t1_std = seasonal_data['t1'].std()
        t2_std = seasonal_data['t2'].std()
        
        for i in gen_data[gen_data['season'] == season].index:
            if np.random.randint(2) == 1:
                gen_data['hum'].values[i] += hum_std/10
            else:
                gen_data['hum'].values[i] -= hum_std/10
                
            if np.random.randint(2) == 1:
                gen_data['wind_speed'].values[i] += wind_speed_std/10
            else:
                gen_data['wind_speed'].values[i] -= wind_speed_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t1'].values[i] += t1_std/10
            else:
                gen_data['t1'].values[i] -= t1_std/10
                
            if np.random.randint(2) == 1:
                gen_data['t2'].values[i] += t2_std/10
            else:
                gen_data['t2'].values[i] -= t2_std/10

    return gen_data

In [21]:
gen = data_enhancement(data)
print(gen.head(3) )

   cnt        t1       t2        hum  wind_speed  weather_code  is_holiday  \
0  182  3.379372  1.51169  91.910483    6.890895           3.0         0.0   
1  138  3.379372  2.98831  94.089517    5.890895           1.0         0.0   
2  134  2.879372  2.01169  97.589517    0.890895           1.0         0.0   

   is_weekend  season  year month hour  
0         1.0     3.0  2015    04   00  
1         1.0     3.0  2015    04   01  
2         1.0     3.0  2015    04   02  


In [22]:
y = data['cnt']
x = data.drop(['cnt'], axis=1)

In [23]:
cat_vars = ['season','is_weekend','is_holiday','year','month','weather_code']
num_vars = ['t1','t2','hum','wind_speed']

In [24]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.2, random_state=0  )

In [25]:
extra_sample = gen.sample(gen.shape[0] // 3)

In [26]:
x_train = pd.concat([x_train, extra_sample.drop(['cnt'], axis=1 ) ])
y_train = pd.concat([y_train, extra_sample['cnt'] ])

In [27]:
transformer = preprocessing.PowerTransformer()
y_train = transformer.fit_transform(y_train.values.reshape(-1,1))
y_val = transformer.transform(y_val.values.reshape(-1,1))

In [37]:
rang = abs(y_train.max()) + abs(y_train.min())

In [ ]:
rang = abs(y_train.max()) + abs(y_train.min())

In [28]:
num_4_treeModels = pipeline.Pipeline(steps=[ ('imputer', impute.SimpleImputer(strategy='constant', fill_value=-9999)),])

In [29]:
cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder()) # handle_unknown='ignore' ONLY IN VERSION 0.24
])

In [30]:
tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

In [31]:
tree_classifiers = {
                        "Decision Tree": DecisionTreeRegressor(),
                        "Extra Trees":   ExtraTreesRegressor(n_estimators=100),
                        "Random Forest": RandomForestRegressor(n_estimators=100),
                        "AdaBoost":      AdaBoostRegressor(n_estimators=100),
 
                    }

In [32]:
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

In [35]:
results = pd.DataFrame({'Model': [], 'MSE': [], 'MAB': [], " % error": [], 'Time': []})

In [38]:
for model_name, model in tree_classifiers.items():
    
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
        
    pred = model.predict(x_val)
    
    results = results.append({"Model":      model_name,
                              "MSE":        metrics.mean_squared_error(y_val, pred),
                              "MAB":        metrics.mean_absolute_error(y_val, pred),
                              " % error":   metrics.mean_squared_error(y_val, pred) / rang,
                              "Time":       total_time},
                              ignore_index=True)

C:\Users\zorve\AppData\Local\Temp\ipykernel_7020\1877558101.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":      model_name,
c:\Users\zorve\anaconda3\envs\deeplearning\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\zorve\AppData\Local\Temp\ipykernel_7020\1877558101.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":      model_name,
c:\Users\zorve\anaconda3\envs\deeplearning\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

In [40]:
results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
results_ord.index += 1 

results_ord.style.bar(subset=['MSE', 'MAB'], vmin=0, vmax=100, color='#5fba7d')

,Model,MSE,MAB,% error,Time
1,Extra Trees,0.388295,0.374497,0.070333,4.996055
2,Random Forest,0.428014,0.456925,0.077528,7.657171
3,AdaBoost,0.700791,0.667897,0.126937,0.664845
4,Decision Tree,0.748114,0.521464,0.135509,0.271823


In [41]:
print(results_ord)

           Model       MSE       MAB   % error      Time
1    Extra Trees  0.388295  0.374497  0.070333  4.996055
2  Random Forest  0.428014  0.456925  0.077528  7.657171
3       AdaBoost  0.700791  0.667897  0.126937  0.664845
4  Decision Tree  0.748114  0.521464  0.135509  0.271823


In [42]:
print(y_train.max())
print(y_train.min())
print(y_val[3])
print(tree_classifiers['Random Forest'].predict(x_val)[3])

2.766201853700867
-2.754580496720054
[0.56668132]
1.010050044253681
